<a href="https://colab.research.google.com/github/Ouasfi/PhilosophIA/blob/master/philosophAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation using AWD_LSTM model 


### Data preparation

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
cd Philosophie_generator/

/gdrive/My Drive/Colab Notebooks/Philosophie_generator


In [0]:
cd My\ Drive/Colab\ Notebooks


[Errno 2] No such file or directory: 'My Drive/Colab Notebooks'
/gdrive/My Drive/Colab Notebooks


In [0]:
from fastai import *
from fastai.text import * 

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
df = pd.read_csv('data/quotes.csv', sep=r'\<\|\>', header=None, names=['text', 'author'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df.head()

,text,author
0,A serious and good philosophical work could be...,Wittgenstein
1,The limits of my language means the limits of ...,Wittgenstein
2,"I don't know why we are here, but I'm pretty s...",Wittgenstein
3,"Whereof one cannot speak, thereof one must be ...",Wittgenstein
4,Hell isn't other people. Hell is yourself.,Wittgenstein


In [0]:
df.describe()

,text,author
count,5130,5130
unique,5114,7
top,To confine our attention to terrestrial matter...,Nietzsche
freq,2,2063


In [0]:
df['text'] = df['text'].str.lower()

In [0]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [0]:
len(train_df), len(valid_df)

(4873, 257)

In [0]:
nan_rows = df[df['text'].isnull()]

In [0]:

nan_rows

,text,author


In [0]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='text',
                           label_cols='author')

In [0]:
# data_clas = TextClasDataBunch.from_df('data', train_df, valid_df, text_cols='text', label_cols='author', vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
help(TextLMDataBunch)

## Model definition and trainig

In [0]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.088228,4.570608,0.225818,00:15


In [0]:
learn.predict("I need", n_words=30, temperature=0.8).replace("xxbos","")


'I need to speak with a person who does not believe in nothing .  My readers are asked to keep . it is what i do , however , and'

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.541966,4.302832,0.255952,00:19


In [0]:
wd=1e-7
lr=1e-3
lrs = lr

In [0]:
learn.fit(10,lrs, wd)

epoch,train_loss,valid_loss,accuracy,time
0,4.318624,4.187274,0.266518,00:19
1,4.165323,4.116737,0.269196,00:19
2,4.024391,4.058914,0.278720,00:19
3,3.861655,4.018850,0.285119,00:19
4,3.702783,3.984146,0.286830,00:19
5,3.546835,3.963060,0.297619,00:19
6,3.401809,3.952606,0.299405,00:19
7,3.251131,3.939972,0.305804,00:19
8,3.102335,3.926567,0.313244,00:19
9,2.977050,3.927818,0.316518,00:19


In [0]:
learn.predict("My ", n_words=15, temperature=0.8).replace("xxbos","")


'My  love for excess is what is most important about happiness : it is a burden'

In [0]:
from random import *
import re

In [0]:
def update_init_w(idea, previous_word):
  
  words = idea.split(" ")
  
  for k in range(0, len(words)):
    i = randint(0, len(words)-1)
    word = words[i]
    if len(word) > 4 and word != previous_word:
        return  word

In [0]:
def text_from_quotes ( init_word, n_words, temp, num_of_ideas):
  number_of_ideas = num_of_ideas
  ideas_counter = 0
  all_ideas = []
  inti_word = init_word

  for i in range(number_of_ideas):
    
      
      idea = learn.predict(init_word, n_words=n_words, temperature=temp)
      idea = idea.replace("xxbos","")
      idea.replace(" . ", ". ").replace(".  ", ". ")
      idea.replace(" .", ". ").replace(".  ", ". ")
      all_ideas.append(idea)
      previous_word = init_word 
      init_word = update_init_w(idea, previous_word)
      
  return all_ideas 

In [0]:
def IA_writing(filename, all_ideas):
  quotes = open(filename, 'w')
  for idea in all_ideas :
    quotes.write(idea)
  quotes.close()
  
  
  

In [0]:
all_ideas = text_from_quotes ( 'simple', 15, 0.8, 10)
IA_writing('IA_philosophy.odt', all_ideas)

In [0]:
learn.save_encoder('ft_enc')

In [0]:
train_df.to_pickle('data/train_df.pkl')

In [0]:
valid_df.to_pickle('data/valid_df.pkl')


In [0]:
#learn = load_learner('./','ft_enc.pth')#

In [0]:
#learn = learn.load('ft_enc')

In [0]:
ls

 data/                     IA_philosophy.odt    'quotes (1).gdoc'   README.md
'Generate text .ipynb'     litterature.txt       quotes.gdoc        w.txt
'IA_philosophy (1).gdoc'   models/               quotes.odt
 IA_philosophy.gdoc        philosophy-AI.ipynb   quotes.txt


nothing is more profound than the stirring of hope .it is generally known , however , that a man has no greatness .patience is inevitable .at the heart of all beauty lies something absurd and solitary .for the most part , little is known about what the world is or where it lives .a sick conscience or a bad conscience might be offered to you as a prison .a true world is a fairy story for people .yes , i understand the moral , as the most natural and the most beautiful form of lifeif you believe in the idea of the self ? what is this capable of going beyond this ?i am too much in love with myself .i say that to know that we are capable of turning